### This notebook demonstrates the process for analyzing the ability to predict whether change from $year_{i}$ to $year_{i+1}$ is consistent using the properties of neighboring pixels (sometimes referred to as the Francis Method). 
The definition of consistent change is up to the user, but one example of consistent change is if from $year_{i}$ to $year_{i+1}$ there was change, and from $year_{i+1}$ to $year_{i+2}$ the state stays the same.

    
To run this notebook, you will need:
1. An image collection of land cover classifications for each time period (e.g. year). The classification images should be images with one band representing the classificaton for that time period.
    * You can squash scene by scene predictions to annual
    * Or have the final classifications for each year already processed
2. An area of interest over which to predict land cover change (e.g. global versus one country like Brazil)


    
To more formally describe the Francis method: 
* The model is predicting whether change from $year_{i}$ to $year_{i+1}$ is consistent, given properties of the neighboring pixels 
* The model is a binary classification model
* The inputs are properties of the surrounding neighborhood, for example how many of the surrounding pixels transitioned, how many of the surroudning pixels are of each class, etc
* Neighborhoods are defined using kernels, and there are many options for the kernel including shape and size
* The output of the model is a probability ranging from 0-1 that the transition is consistent

The steps of this notebook are as follows:
1. Define consistent change
2. Define predictor variables 
3. Define kernels to test different neighborhoods
4. Gather a training set, validation set, and test set
    * For simplicity we will only look at one year of change.
5. Define models we will test (e.g logistic regression, random forest, maxEntropy)
6. Use cross validation to choose optimal parameters for each model, kernel option
7. Train a model with the optimal parameters for each model, kernel option and compare on validation set to choose the best model.
8. Use model to predict consistent change


## Step 0: Load libraries and iniatilize Earth Engine

In [ ]:
#Load necessary libraries
import sys

sys.path.append('/usr/local/lib/python3.8/site-packages')
sys.path.append('/Users/kristine/Library/Python/3.8/lib/python/site-packages')
import os
import ee
import geemap
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from ipyleaflet import Map, basemaps
import random
import json
import time
import ast

# relative import for this folder hierarchy, credit: https://stackoverflow.com/a/35273613
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)

from wri_change_detection import preprocessing as npv
from wri_change_detection import gee_classifier as gclass



<font size="4">Iniatilize Earth Engine and Google Cloud authentication</font>

In [ ]:
#Initialize earth engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

<font size="4">Define a seed number to ensure reproducibility across random processes. This seed will be used in all subsequent sampling as well. We'll also define seeds for sampling the training, validation, and test sets.</font>

In [ ]:
num_seed=30
random.seed(num_seed)


## Step 1: Load Land Cover Classifications and Define Consistent Change


<font size="4">We will start by gathering monthly land cover probabilities and squashing them to annual classifications by taking the median.
Define land cover classification image collection, with one image for each time period. Each image should have one band representing the classification in that pixel for one time period.</font>

In [ ]:
#Load collection
#This collection represents monthly dynamic world classifications of land cover, later we'll squash it to annual
dynamic_world_classifications_monthly = ee.ImageCollection('projects/wings-203121/assets/dynamic-world/v3-5_stack_tests/wri_test_goldsboro')

#Get classes from first image
dw_classes = dynamic_world_classifications_monthly.first().bandNames()
dw_classes_str = dw_classes.getInfo()

#Get dictionary of classes and values
#Define array of land cover classification values
dw_class_values = np.arange(1,10).tolist()
dw_class_values = ee.List(dw_class_values)
#Create dictionary representing land cover classes and land cover class values
dw_classes_dict = ee.Dictionary.fromLists(dw_classes, dw_class_values)


<font size="4">Define color palettes to map land cover</font>

In [ ]:
change_detection_palette = ['#ffffff', # no_data=0
                              '#419bdf', # water=1
                              '#397d49', # trees=2
                              '#88b053', # grass=3
                              '#7a87c6', # flooded_vegetation=4
                              '#e49535', # crops=5
                              '#dfc25a', # scrub_shrub=6
                              '#c4291b', # builtup=7
                              '#a59b8f', # bare_ground=8
                              '#a8ebff', # snow_ice=9
                              '#616161', # clouds=10
]
statesViz = {'min': 0, 'max': 10, 'palette': change_detection_palette};


<font size="4">Gather projection and geometry information from the land cover classifications</font>

In [ ]:
projection = dynamic_world_classifications_monthly.first().projection().getInfo()
crs = projection.get('crs')
crsTransform = projection.get('transform')
scale = dynamic_world_classifications_monthly.first().projection().nominalScale().getInfo()
print('CRS and Transform: ',crs, crsTransform)

geometry = dynamic_world_classifications_monthly.first().geometry().bounds()


<font size="4">Define years to assess land cover and convert monthly classifications to annual classifications</font>

In [ ]:
#Define years to get annual classifications for
years = np.arange(2016,2020)

#Squash scenes from monthly to annual for both the annual classification and annual probabilities
dynamic_world_classifications = npv.squashScenesToAnnualClassification(dynamic_world_classifications_monthly,years,method='median')
dynamic_world_annual_probabilities = npv.squashScenesToAnnualProbability(dynamic_world_classifications_monthly,years,method='median',image_prefix='prob_')


<font size="4">Define consistent change image to sample training points. Here we will define consistent change as:
* $$\ state(i) != state(i+1) $$
* $$\ state(i+1) = state(i+2) $$
</font>

In [ ]:
#Convert classifications to an image with band names as original image names
dynamic_world_classification_image = dynamic_world_classifications.toBands()

#Get a collection of images where each image has 3 bands: classifications for year(i), classifications for year(i+1), and classifications for year i+2
lc_consistent_change_one_year_col = npv.getYearStackIC(dynamic_world_classification_image,dynamic_world_classification_image.bandNames().getInfo(), band_indices=[0,1,2])

#Get a collection of images where each image represents whether change was consistent from year(i) to year(i+1)
lc_consistent_change_one_year_col = lc_consistent_change_one_year_col.map(npv.LC_ConsistentChangeOneYear)

#Convert this collection to one image
lc_consistent_change_one_year_image = lc_consistent_change_one_year_col.toBands()

#Rename image bands to the central year
lc_consistent_change_one_year_image = lc_consistent_change_one_year_image.rename(lc_consistent_change_one_year_col.aggregate_array('OriginalBand'))



<font size="4">Now *lc_consistent_change_one_year_image* is a binary multi-band image where each band represents
whether change from $year_{i}$ to $year_{i+1}$ is consistent

However it also includes 0's for where there was no change from $year_{i}$ to $year_{i+1}$, therefore we need to
mask the *lc_consistent_change_one_year_image* to only pixels with change form $year_{i}$ to $year_{i+1}$
</font>

<font size="4">Mask consistent change image to pixels with change in that year</font>

In [ ]:
#Get a collection of images where each image has 2 bands: classifications for year(i) and classifications for year(i+1)
lc_one_change_col = npv.getYearStackIC(dynamic_world_classification_image,dynamic_world_classification_image.bandNames().getInfo(), band_indices=[0,1])

#Get a collection of images where each image represents whether there was change from year(i) to year(i+1)
lc_one_change_col = lc_one_change_col.map(npv.LC_OneChange)

#Convert this collection to one image
lc_one_change_image = lc_one_change_col.toBands()

#Rename image bands to the central year
lc_one_change_image = lc_one_change_image.rename(lc_one_change_col.aggregate_array('OriginalBand'))

#Mask each band of the consistent change image to 
lc_consistent_change_one_year_image = lc_consistent_change_one_year_image.mask(lc_one_change_image.select(lc_consistent_change_one_year_image.bandNames()))


<font size="4">Map three years worth of land cover and the consistent change image to check *lc_consistent_change_one_year_image*
</font>

In [ ]:
oneChangeDetectionViz = {'min': 0, 'max': 1, 'palette': ['696a76','ff2b2b']}; #gray = 0, red = 1
consistentChangeDetectionViz = {'min': 0, 'max': 1, 'palette': ['0741df','df07b5']}; #blue = 0, pink = 1


center = [35.410769, -78.100163]
zoom = 12
Map1 = geemap.Map(center=center, zoom=zoom,basemap=basemaps.Esri.WorldImagery,add_google_map = False)
Map1.addLayer(dynamic_world_classification_image.select('2017_class'),statesViz,name='2017 LC')
Map1.addLayer(dynamic_world_classification_image.select('2018_class'),statesViz,name='2018 LC')
Map1.addLayer(dynamic_world_classification_image.select('2019_class'),statesViz,name='2019 LC')
Map1.addLayer(lc_one_change_image.select('2017_class'),oneChangeDetectionViz,name='One Change from 2017-2018')
Map1.addLayer(lc_consistent_change_one_year_image.select('2017_class'),consistentChangeDetectionViz,
              name='Consistent Change from 2017-2019')
display(Map1)


* The first layer will show classifications for 2017
* The second layer will show classifications for 2018
* The third layer will show classifications for 2019
* The fourth layer will show whether change occured from 2017 to 2018, with grey for no and red for yes
* The fifth layer will show whether change from 2017 to 2018 stayed consistent in 2019, with blue for no and pink for yes
* The final layer should be masked to show only consistent change in the red areas of the fourth layer


## Step 2: Gather neighborhood predictor variables 

In [ ]:
#Apply land cover change functions to images, first returns an image collection then converted to image
#where each band represents one year

#Get band names
dwc_bandNames = dynamic_world_classification_image.bandNames().getInfo()

#Pixels where state(i-1) != state(i) and state(i-1) = state(i+1)
lc_reversed_col = npv.getYearStackIC(dynamic_world_classification_image, dwc_bandNames, band_indices=[-1,0,1])
lc_reversed_col = lc_reversed_col.map(npv.LC_Reverse)
lc_reversed_image = lc_reversed_col.toBands()
lc_reversed_image = lc_reversed_image.rename(lc_reversed_col.aggregate_array('OriginalBand'))
lc_reversed_image = lc_reversed_image.set('system:index','LC_Reversed')

#Pixels where state(i-1) != state(i) and state(i) != state(i+1) and state(i-1) != state(i+1)
lc_changed_to_another_col = npv.getYearStackIC(dynamic_world_classification_image, dwc_bandNames,band_indices=[-1,0,1])
lc_changed_to_another_col = lc_changed_to_another_col.map(npv.LC_ChangeToAnother)
lc_changed_to_another_image = lc_changed_to_another_col.toBands()
lc_changed_to_another_image = lc_changed_to_another_image.rename(lc_changed_to_another_col.aggregate_array('OriginalBand'))
lc_changed_to_another_image = lc_changed_to_another_image.set('system:index','LC_ChangedToAnother')


#Get binary images of the land cover classifications for the following year
lc_year_after_col = npv.getYearStackIC(dynamic_world_classification_image, dwc_bandNames, band_indices=[0,1])
lc_year_after_col = lc_year_after_col.map(npv.LC_YearAfter)
lc_year_after_image = lc_year_after_col.toBands()
lc_year_after_image = lc_year_after_image.rename(lc_year_after_col.aggregate_array('OriginalBand'))
lc_year_after_image = npv.convertClassificationsToBinaryImages(lc_year_after_image, dw_classes_dict)

#Get binary images of the land cover classifications for the current year
lc_year_image = npv.convertClassificationsToBinaryImages(dynamic_world_classification_image, dw_classes_dict)



Now we have mutliple options for predictor variables including:
1. How many pixels from $year_{i-1}$ to $year_{i}$ changed, then from $year_{i}$ to $year_{i+1}$ flipped back to $state_{i-1}$
2. How many pixels from $year_{i-1}$ to $year_{i}$ changed, then from $year_{i}$ to $year_{i+1}$ changed to a different state than $state_{i-1}$
3. How many pixels changed from $year_{i}$ to $year_{i+1}$
4. How many pixels are of each class in $year_{i}$
5. How many pixels are of each class in $year_{i+1}$
6. The average probability of each class in $year_{i}$
7. The average probability of each class in $year_{i+1}$


<font size="4">For the remainder of the notebook, we'll sample training points and predictor variables for one year only.

Gather predictor variables for 2017, and build a list of predictor variables and defined column names</font>

In [ ]:
year_to_select = '2017_class'
year_after_to_select = '2018_class'
lc_year_after_image_select = lc_year_after_image.filterMetadata('system:index','equals',year_to_select).first()
lc_year_image_select = lc_year_image.filterMetadata('system:index','equals',year_to_select).first()

current_year_probs = dynamic_world_annual_probabilities.filterMetadata('system:index','equals','prob_2017').first()
current_year_probs = current_year_probs.set('system:index','current_year_prob')

following_year_probs = dynamic_world_annual_probabilities.filterMetadata('system:index','equals','prob_2018').first()
following_year_probs = current_year_probs.set('system:index','following_year_prob')

#We'll only select the "one_change", "reversed", "changed to another", probabilities for the current year,
#and probabilities for the following year as the predictor variables to test
predictor_variable_list = [lc_one_change_image.select(year_to_select),
                           lc_reversed_image.select(year_to_select),
                           lc_changed_to_another_image.select(year_to_select),
                           current_year_probs,
                           following_year_probs]
predictor_variable_image = ee.ImageCollection(predictor_variable_list).toBands()

#Rename bands to more intelligible ones
predictor_variable_names = ['LC_OneChange','LC_Reversed','LC_ChangedToAnother']+['current_year_{}'.format(x) for x in current_year_probs.bandNames().getInfo()]+['following_year_{}'.format(x) for x in following_year_probs.bandNames().getInfo()]
predictor_variable_image = predictor_variable_image.rename(predictor_variable_names)

# #If you want to select the "how many pixels of each class" for the current year and following year, you can use the line below to rename predictor_variable_image for those images
# #['current_year_{}'.format(x) for x in dw_classes_dict.keys().getInfo()]+['following_year_{}'.format(x) for x in dw_classes_dict.keys().getInfo()]



## Step 3: Define Kernels to Test

In [ ]:

#Here are are some sample kernels you can try, we'll be testing the performance of each of them
fixed_kernel = ee.Kernel.fixed(3,3,
                         [[1,1,1],
                          [1,0,1],
                          [1,1,1]]
                          ,1,1)

gaussian_kernel = ee.Kernel.gaussian(radius=1000, units='meters', sigma=1000)

circle_kernel = ee.Kernel.circle(radius=17, units='pixels')

square_kernel = ee.Kernel.square(radius=1.5, units='pixels')


#Convolve the predictor variable image to get the neighborhood values of each predictor variable
fixed_kernel_predictors = predictor_variable_image.convolve(fixed_kernel)
gaussian_kernel_predictors = predictor_variable_image.convolve(gaussian_kernel)
circle_kernel_predictors = predictor_variable_image.convolve(circle_kernel)
square_kernel_predictors = predictor_variable_image.convolve(square_kernel)

#Now to easily loop over the kernels through this process, we'll put them into a dictionary. 
kernel_dictionary = {
    'fixed_kernel':fixed_kernel_predictors,
    'gaussian_kernel': gaussian_kernel_predictors,
    'circle_kernel':circle_kernel_predictors,
    'square_kernel':square_kernel_predictors,
}


## Step 4: Gather Training Points

<font size="4">Now that we have a binary image of where consistent change occurs (masked to areas of change) and our convolved predictor variables, we'll build our training, validation, and test set. We'll only sample training points from one year to simplify the process, however you can sample locations from multiple years.

First we'll sample the training point locations by taking stratified samples of the consistent change image, then we'll sample the predictor variables at those locations.</font>

In [ ]:
#We'll remove training points that are within a certain distance to the validation and test points
#However since we're testing over a small area, we'll keep the distance small
distanceToFilter = 100


#Select consistent change for 2017
lc_consistent_change_one_year_select = lc_consistent_change_one_year_image.select(year_to_select).rename('Consistent Change')

#Sample point locations that we will split into training, validation, and test sets
point_locations = npv.getStratifiedSampleBandPoints(lc_consistent_change_one_year_select, region=geometry, 
                                                       numPoints=2000, bandName='Consistent Change',seed=num_seed,
                                                       geometries=True,scale=scale, projection=crs)

#Assign random value between 0 and 1 to split into training, validation, and test sets
point_locations = point_locations.randomColumn(columnName='TrainingSplit', seed=num_seed)
#First split training set from the rest, taking 70% of the points for training
#Roughly 70% training, 30% for validation + testing
training_split = 0.7
training_locations = point_locations.filter(ee.Filter.lt('TrainingSplit', training_split))
validation_and_test = point_locations.filter(ee.Filter.gte('TrainingSplit', training_split))

#Define distance filter to remove training points within a certain distance of test points and validation points
distFilter = ee.Filter.withinDistance(distance=distanceToFilter, leftField='.geo', rightField= '.geo', maxError= 1)
join = ee.Join.inverted()
training_locations = join.apply(training_locations, validation_and_test, distFilter);

#Assign another random value between 0 and 1 to validation_and_test to split to validation and test sets
validation_and_test = validation_and_test.randomColumn(columnName='ValidationSplit', seed=num_seed)
#Of the 30% saved for validation + testing, half goes to validation and half goes to test
#Meaning original sample will be 70% training, 15% validation, 15% testing
validation_split = 0.5 
validation_locations = validation_and_test.filter(ee.Filter.lt('ValidationSplit', validation_split))
test_locations = validation_and_test.filter(ee.Filter.gte('ValidationSplit', validation_split))

#Apply distance filter to remove validation points within a certain distance of test points
validation_locations = join.apply(validation_locations, test_locations, distFilter);

#Export these locations to an Earth Engine asset
location_description = '{}_locations'
location_assetID = 'projects/wri-datalab/DynamicWorld_CD/ModelResults/Neighborhood_{}_locations'

export_results_task = ee.batch.Export.table.toAsset(
    collection=training_locations, 
    description = location_description.format('training'), 
    assetId = location_assetID.format('training'))
export_results_task.start()
    
export_results_task = ee.batch.Export.table.toAsset(
    collection=validation_locations, 
    description = location_description.format('validation'), 
    assetId = location_assetID.format('validation'))
export_results_task.start()

export_results_task = ee.batch.Export.table.toAsset(
    collection=test_locations, 
    description = location_description.format('test'), 
    assetId = location_assetID.format('test'))
export_results_task.start()
 
#Wait for last export to finish
while export_results_task.active():
    print('Polling for task (id: {}).'.format(export_results_task.id))
    time.sleep(30)
print('Done with export.')


<font size="4">Next we'll sample the neighborhood predictor variables for each of the neighborhoods for the training, validation, and test sets.</font>

In [ ]:
#Define properties to export to an Earth Engine asset
points_description = '{}_{}_points'
points_assetID = 'projects/wri-datalab/DynamicWorld_CD/ModelResults/Neighborhood_{}_{}_points'

#Load the point locations assets
training_locations_asset = ee.FeatureCollection('projects/wri-datalab/DynamicWorld_CD/ModelResults/Neighborhood_{}_locations'.format('training'))
validation_locations_asset = ee.FeatureCollection('projects/wri-datalab/DynamicWorld_CD/ModelResults/Neighborhood_{}_locations'.format('validation'))
test_locations_asset = ee.FeatureCollection('projects/wri-datalab/DynamicWorld_CD/ModelResults/Neighborhood_{}_locations'.format('test'))


#Iterate through kernel dictionary
for key,value in kernel_dictionary.items():
    #Sample the neighborhood image at point locations 
    training_points = value.sampleRegions(training_locations_asset, scale=scale, projection=crs, geometries=True,tileScale=2)
    validation_points = value.sampleRegions(validation_locations_asset, scale=scale, projection=crs, geometries=True,tileScale=2)
    test_points = value.sampleRegions(test_locations_asset, scale=scale, projection=crs, geometries=True,tileScale=2)
    
    export_results_task = ee.batch.Export.table.toAsset(
        collection=training_points, 
        description = points_description.format(key,'training'), 
        assetId = points_assetID.format(key,'training'))
    export_results_task.start()
    
    export_results_task = ee.batch.Export.table.toAsset(
        collection=validation_points, 
        description = points_description.format(key,'validation'), 
        assetId = points_assetID.format(key,'validation'))
    export_results_task.start()
    
    export_results_task = ee.batch.Export.table.toAsset(
        collection=test_points, 
        description = points_description.format(key,'test'), 
        assetId = points_assetID.format(key,'test'))
    export_results_task.start()
    
    
    
    
    

## Step 5: define models to test

In [ ]:
#Define dictionaries of parameters and models to test
#You can find the inputs for the parameters under the ee.Classifiers section of GEE

rf_parameters = {'seed':[num_seed], 
          'numberOfTrees': [50,100], 
          'variablesPerSplit': [4,8,10,None], 
          'minLeafPopulation': [None,10,50], 
          'bagFraction': [None,0.5,.3], 
          'maxNodes': [None, 20, 50]
         }
#buildGridSearchList converts the parameter dictionary into a list of classifiers that can be used in cross-validation
rf_classifier_list = gclass.buildGridSearchList(rf_parameters,'smileRandomForest')

svm_parameters = {'decisionProcedure':[None]}
svm_classifier_list = gclass.buildGridSearchList(svm_parameters,'libsvm')

maxent_parameters = {'minIterations':[10,100],
                    'maxIterations':[50,200]}
maxent_classifier_list = gclass.buildGridSearchList(maxent_parameters,'gmoMaxEnt')

classifier_list = rf_classifier_list+svm_classifier_list+maxent_classifier_list



## Step 6: Use cross validation to choose optimal parameters for each model, kernel option


In [ ]:
#Name y_column
y_column = 'Consistent Change'

#Define assetId and description format to export to GEE
cv_results_assetId = 'projects/wri-datalab/DynamicWorld_CD/ModelResults/cv_export_{}'
cv_results_description = 'cv_export_{}'

#Loop through kernel_dictionary to perform cross validation
for key, value in kernel_dictionary.items():
    #Try statement only because the training set export takes a while for some neighborhoods
    try:
        #Load training points
        training_points = ee.FeatureCollection(points_assetID.format(key,'training'))

        #Perform cross validation, returns a feature collection
        cv_results = gclass.kFoldCrossValidation(inputtedFeatureCollection = training_points, 
                                             propertyToPredictAsString = y_column, 
                                             predictors = predictor_variable_names, 
                                             listOfClassifiers = classifier_list,
                                             k=3,seed=num_seed)
        #Export results to GEE
        export_results_task = ee.batch.Export.table.toAsset(
                collection=cv_results, 
                description = cv_results_description.format(key), 
                assetId = cv_results_assetId.format(key))
        export_results_task.start()
    except:
        None



<font size="4">Load cross validation results and find the best model based on accuracy on the validation set</font>

In [ ]:
#Create empty dataframe to save results of cross validation
accuracy_and_keys = pd.DataFrame()

#Loop through kernels
for key, value in kernel_dictionary.items():
    
    #Load cross-validation results
    results = ee.FeatureCollection(cv_results_assetId.format(key))
    #Get the best result by the cross validation score
    best_result = results.sort('Validation Score', False).first()
    
    #Load params as a dictionary
    params = best_result.get('Params').getInfo()
    params = ast.literal_eval(params)
    
    #Get the calssifier name
    classifierName = best_result.get('Classifier Type').getInfo()
    
    #Load classifier with best params
    classifier = gclass.defineClassifier(params,classifierName)

    #Load training and validation points
    training_points = ee.FeatureCollection(points_assetID.format(key,'training'))
    validation_points = ee.FeatureCollection(points_assetID.format(key,'validation'))
    
    #Train a classifier with the best params on the training data
    classifier = classifier.train(training_points, classProperty=y_column, 
                                  inputProperties=predictor_variable_names, subsamplingSeed=num_seed)
    
    #Predict over validation data
    validation_points_predicted = validation_points.classify(classifier)
    
    #Get confusion matrix and accuracy score
    confusion_matrix = validation_points_predicted.errorMatrix(y_column, 'classification');
    accuracy = confusion_matrix.accuracy().getInfo()

    #Save results to dataframe
    results_dict = {'key':key,'accuracy':accuracy,'Params':params,'Classifier Type':classifierName}
    accuracy_and_keys = accuracy_and_keys.append(results_dict, ignore_index=True)
    
    #Print results
    print('Kernel Name',key, 'Classifier',classifierName)
    print(gclass.pretty_print_confusion_matrix(confusion_matrix.getInfo()))
    print('Accuracy',accuracy)
    print('\n')

    
#Get the best model
best_model_row = accuracy_and_keys.sort_values(by=['accuracy'],ascending=False).iloc[0]
#Set it up as a GEE Classifier
best_model = gclass.defineClassifier(best_model_row['Params'],best_model_row['Classifier Type'])
best_model = best_model.train(training_points, classProperty=y_column, 
                                  inputProperties=predictor_variable_names, subsamplingSeed=num_seed)
#Predict over validation data
validation_points_predicted = validation_points.classify(best_model)

#Get confusion matrix and accuracy score
confusion_matrix = validation_points_predicted.errorMatrix(y_column, 'classification');
accuracy = confusion_matrix.accuracy().getInfo()
print('--------------------------------------------------------------')
print('Final Model Validation Set Confusion Matrix')
print(gclass.pretty_print_confusion_matrix(confusion_matrix.getInfo()))
print('Final Model Validation Set Accuracy',accuracy)
print('\n')


#Predict over test data
test_points = ee.FeatureCollection(points_assetID.format(key,'test'))

test_points_predicted = test_points.classify(best_model)

#Get confusion matrix and accuracy score
confusion_matrix = test_points_predicted.errorMatrix(y_column, 'classification');
accuracy = confusion_matrix.accuracy().getInfo()
print('--------------------------------------------------------------')
print('Final Model Test Set Confusion Matrix')
print(gclass.pretty_print_confusion_matrix(confusion_matrix.getInfo()))
print('Final Model Test Set Accuracy',accuracy)
print('\n')


## Step 8: Use model to predict consistent change

In [ ]:
#Use best model and predictor variable image to predict consistent change
predicted_consistent_change = predictor_variable_image.updateMask(lc_one_change_image.select(year_to_select)).classify(best_model)

#The actual consistent change was saved to a variable lc_consistent_change_one_year_select earlier

#Map the results
center = [35.410769, -78.100163]
zoom = 12
Map2 = geemap.Map(center=center, zoom=zoom,basemap=basemaps.Esri.WorldImagery,add_google_map = False)
Map2.addLayer(dynamic_world_classification_image.select('2017_class'),statesViz,name='2017 LC')
Map2.addLayer(dynamic_world_classification_image.select('2018_class'),statesViz,name='2018 LC')
Map2.addLayer(dynamic_world_classification_image.select('2019_class'),statesViz,name='2019 LC')
Map2.addLayer(lc_consistent_change_one_year_select,consistentChangeDetectionViz,
              name='Consistent Change from 2017-2019')
Map2.addLayer(predicted_consistent_change,oneChangeDetectionViz,
              name='Predicted Consistent Change from 2017-2018')

display(Map2)


#### * The first layer will show classifications for 2017
#### * The second layer will show classifications for 2018
#### * The third layer will show classifications for 2019
#### * The fourth layer will show whether change from 2017 to 2018 stayed consistent in 2019, with grey for no and red for yes with blue for no and pink for yes
#### * The fifth layer will show the model's prediction on whether change from 2017 to 2018 stayed consistent in 2019, blue for no and pink for yes


In [ ]:
#You can view/save the model using the "explain" function of classifiers
classifierString = best_model.explain().get('trees')
print(classifierString.getInfo())